In [26]:
import numpy as np
import pandas as pd
import scipy
import sklearn

In [89]:
import warnings
warnings.filterwarnings('ignore')

## Read in data

In [231]:
train = pd.read_csv('../Data/data_NESARC_tr.csv', header = None)
train.columns = [indices['indices'].values.tolist()]

In [232]:
indices = pd.read_csv('../Data/indices_NESARC.csv', header = None, names = ['start','end'])
indices['indices'] = indices.start.astype(str).str.cat(indices.end.astype(str), sep='-')

In [233]:
target = train.ix[:,'313-313']
target.name = 'DRANK AT LEAST 1 ALCOHOLIC DRINK IN LIFE'
target = target.map({1:0,2:1})

In [234]:
useful_col = ['42-42', '48-49', '63-63', '64-64', '68-69', '79-79', '3673-3674', '114-114', '131-132', '136-136',
              '144-144', '163-163', '196-197', '217-217', '226-226', '230-230', '232-232', '294-294', '306-307', '308-309'
             ,'310-312']

raw = train.ix[:, useful_col]

## Clean the data

In [235]:
col_names = ['CENSUS DIVISION', 'NUMBER OF PERSONS IN HOUSEHOLD', 'FATHER OF RESPONDENT IN HOUSEHOLD', 
             'MOTHER OF RESPONDENT IN HOUSEHOLD', 'AGE', 'SEX', 'IMPUTED RACE/ETHNICITY', 'CURRENT MARITAL STATUS',
            'HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED', 'PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK)',
            'PRESENT SITUATION INCLUDES RETIRED', 'WORK AT JOB OR BUSINESS IN LAST 12 MONTHS', 
             'TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)',
             'PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS', 'CURRENTLY COVERED BY MEDICAID',
             'CURRENTLY COVERED BY PRIVATE HEALTH INSURANCE',
            'SELF-PERCEIVED CURRENT HEALTH','ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS','HEIGHT: FEET', 
             'HEIGHT: INCHES', 'WEIGHT: POUNDS']

In [236]:
raw.columns = col_names

In [237]:
raw.head()

,CENSUS DIVISION,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,IMPUTED RACE/ETHNICITY,CURRENT MARITAL STATUS,HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),...,WORK AT JOB OR BUSINESS IN LAST 12 MONTHS,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS),PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS,CURRENTLY COVERED BY MEDICAID,CURRENTLY COVERED BY PRIVATE HEALTH INSURANCE,SELF-PERCEIVED CURRENT HEALTH,ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS,HEIGHT: FEET,HEIGHT: INCHES,WEIGHT: POUNDS
0,3,2,2,2,41,1,5,4,10,1,...,1,10,2,2,1,2,2,5,7,185
1,5,2,2,2,65,2,1,1,8,2,...,2,13,2,2,1,2,2,5,3,130
2,9,3,2,2,39,1,1,1,8,1,...,1,11,2,2,1,2,1,5,11,195
3,1,2,2,2,49,1,1,4,10,1,...,1,12,2,2,1,2,2,6,0,200
4,2,5,2,1,25,1,1,6,8,1,...,1,10,2,2,2,1,1,5,9,180


In [238]:
## Fill unknowns
raw['ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'] = raw['ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'].replace(9,2)
raw['SELF-PERCEIVED CURRENT HEALTH'] = raw['SELF-PERCEIVED CURRENT HEALTH'].replace(9,3)
raw['HEIGHT: INCHES']  = raw['HEIGHT: INCHES'].replace(99,np.median(raw['HEIGHT: INCHES']))
raw['HEIGHT: FEET']  = raw['HEIGHT: FEET'].replace(99,np.median(raw['HEIGHT: FEET']))

## Combine Heights
raw['HEIGHT'] = raw['HEIGHT: FEET']*12 + raw['HEIGHT: INCHES']

In [120]:
train['294-294'].value_counts()

2    20098
1     9618
9      284
Name: 294-294, dtype: int64

In [121]:
raw['ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'].value_counts()

2    20382
1     9618
Name: ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS, dtype: int64

In [124]:
train['232-232'].value_counts()

1    8612
2    8605
3    7427
4    3647
5    1541
9     168
Name: 232-232, dtype: int64

In [123]:
raw['SELF-PERCEIVED CURRENT HEALTH'].value_counts()

1    8612
2    8605
3    7595
4    3647
5    1541
Name: SELF-PERCEIVED CURRENT HEALTH, dtype: int64

In [239]:
to_be_encoded = ['CENSUS DIVISION', 'IMPUTED RACE/ETHNICITY', 'CURRENT MARITAL STATUS',
                 'HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED', 
                 'TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)'
                ,'SELF-PERCEIVED CURRENT HEALTH', ]
to_be_mapped = ['FATHER OF RESPONDENT IN HOUSEHOLD', 'MOTHER OF RESPONDENT IN HOUSEHOLD', 'SEX', 
               'PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK)','PRESENT SITUATION INCLUDES RETIRED'
               , 'WORK AT JOB OR BUSINESS IN LAST 12 MONTHS', 'PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS'
                ,'CURRENTLY COVERED BY MEDICAID', 'ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'
               ]
to_be_normalized = ['NUMBER OF PERSONS IN HOUSEHOLD','AGE','HEIGHT', 'WEIGHT: POUNDS']

In [240]:
def col_encode(col):
    return pd.get_dummies(raw[col], prefix = col)

In [241]:
for col in to_be_encoded:
    raw = pd.concat([raw, col_encode(col)], axis = 1)

In [242]:
raw.head()

,CENSUS DIVISION,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,IMPUTED RACE/ETHNICITY,CURRENT MARITAL STATUS,HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),...,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_17,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_18,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_19,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_20,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_21,SELF-PERCEIVED CURRENT HEALTH_1,SELF-PERCEIVED CURRENT HEALTH_2,SELF-PERCEIVED CURRENT HEALTH_3,SELF-PERCEIVED CURRENT HEALTH_4,SELF-PERCEIVED CURRENT HEALTH_5
0,3,2,2,2,41,1,5,4,10,1,...,0,0,0,0,0,0,1,0,0,0
1,5,2,2,2,65,2,1,1,8,2,...,0,0,0,0,0,0,1,0,0,0
2,9,3,2,2,39,1,1,1,8,1,...,0,0,0,0,0,0,1,0,0,0
3,1,2,2,2,49,1,1,4,10,1,...,0,0,0,0,0,0,1,0,0,0
4,2,5,2,1,25,1,1,6,8,1,...,0,0,0,0,0,1,0,0,0,0


In [243]:
def col_mapped(col):
    return raw[col].map({1:0,2:1})

In [244]:
for col in to_be_mapped:
    raw[col] = col_mapped(col)

In [245]:
raw.head()

,CENSUS DIVISION,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,IMPUTED RACE/ETHNICITY,CURRENT MARITAL STATUS,HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),...,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_17,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_18,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_19,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_20,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_21,SELF-PERCEIVED CURRENT HEALTH_1,SELF-PERCEIVED CURRENT HEALTH_2,SELF-PERCEIVED CURRENT HEALTH_3,SELF-PERCEIVED CURRENT HEALTH_4,SELF-PERCEIVED CURRENT HEALTH_5
0,3,2,1,1,41,0,5,4,10,0,...,0,0,0,0,0,0,1,0,0,0
1,5,2,1,1,65,1,1,1,8,1,...,0,0,0,0,0,0,1,0,0,0
2,9,3,1,1,39,0,1,1,8,0,...,0,0,0,0,0,0,1,0,0,0
3,1,2,1,1,49,0,1,4,10,0,...,0,0,0,0,0,0,1,0,0,0
4,2,5,1,0,25,0,1,6,8,0,...,0,0,0,0,0,1,0,0,0,0


In [246]:
from sklearn.preprocessing import MinMaxScaler
def col_normalized(col):
    return MinMaxScaler().fit_transform(raw[col].values.reshape(-1,1))

In [247]:
raw['AGE'] = MinMaxScaler().fit_transform(raw['AGE'].values.reshape(-1,1))

In [248]:
for col in to_be_normalized:
    raw[col]  = col_normalized(col)

In [249]:
raw = raw.drop(to_be_encoded, axis = 1)
raw = raw.drop(['HEIGHT: FEET', 'HEIGHT: INCHES'], axis = 1)

In [250]:
raw.head()

,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),PRESENT SITUATION INCLUDES RETIRED,WORK AT JOB OR BUSINESS IN LAST 12 MONTHS,PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS,CURRENTLY COVERED BY MEDICAID,...,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_17,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_18,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_19,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_20,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_21,SELF-PERCEIVED CURRENT HEALTH_1,SELF-PERCEIVED CURRENT HEALTH_2,SELF-PERCEIVED CURRENT HEALTH_3,SELF-PERCEIVED CURRENT HEALTH_4,SELF-PERCEIVED CURRENT HEALTH_5
0,0.0625,1,1,0.2875,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,0.0625,1,1,0.5875,1,1,0,1,1,1,...,0,0,0,0,0,0,1,0,0,0
2,0.1250,1,1,0.2625,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,0.0625,1,1,0.3875,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
4,0.2500,1,0,0.0875,0,0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0


In [251]:
raw['ONES'] = np.ones(raw.shape[0])
raw['TARGET'] = target

In [252]:
raw.head()

,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),PRESENT SITUATION INCLUDES RETIRED,WORK AT JOB OR BUSINESS IN LAST 12 MONTHS,PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS,CURRENTLY COVERED BY MEDICAID,...,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_19,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_20,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_21,SELF-PERCEIVED CURRENT HEALTH_1,SELF-PERCEIVED CURRENT HEALTH_2,SELF-PERCEIVED CURRENT HEALTH_3,SELF-PERCEIVED CURRENT HEALTH_4,SELF-PERCEIVED CURRENT HEALTH_5,ONES,TARGET
0,0.0625,1,1,0.2875,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0,1.0,0
1,0.0625,1,1,0.5875,1,1,0,1,1,1,...,0,0,0,0,1,0,0,0,1.0,0
2,0.1250,1,1,0.2625,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0,1.0,0
3,0.0625,1,1,0.3875,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0,1.0,0
4,0.2500,1,0,0.0875,0,0,1,0,1,1,...,0,0,0,1,0,0,0,0,1.0,0


In [253]:
raw.to_csv('hw1_train.csv')

In [293]:
test = pd.read_csv('../Data/data_NESARC_ts.csv', header = None)

In [296]:
test.columns = [indices['indices'].values.tolist()]
test.shape

(13093, 2991)

In [299]:
test.columns = [indices['indices'].values.tolist()]
useful_col = ['42-42', '48-49', '63-63', '64-64', '68-69', '79-79', '3673-3674', '114-114', '131-132', '136-136','144-144', '163-163', '196-197', '217-217', '226-226', '230-230', '232-232', '294-294', '306-307', '308-309'
             ,'310-312']

rawt = test.ix[:, useful_col]

targett = test.ix[:,'313-313']
targett.name = 'DRANK AT LEAST 1 ALCOHOLIC DRINK IN LIFE'
targett = targett.map({1:0,2:1})

In [300]:
rawt.shape

(13093, 21)

In [301]:
rawt.columns = col_names
rawt.head()

,CENSUS DIVISION,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,IMPUTED RACE/ETHNICITY,CURRENT MARITAL STATUS,HIGHEST GRADE OR YEAR OF SCHOOL COMPLETED,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),...,WORK AT JOB OR BUSINESS IN LAST 12 MONTHS,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS),PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS,CURRENTLY COVERED BY MEDICAID,CURRENTLY COVERED BY PRIVATE HEALTH INSURANCE,SELF-PERCEIVED CURRENT HEALTH,ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS,HEIGHT: FEET,HEIGHT: INCHES,WEIGHT: POUNDS
0,3,3,2,2,77,2,1,3,3,2,...,2,9,2,2,2,1,1,5,5,180
1,9,1,2,2,24,2,1,6,12,1,...,1,8,2,2,1,2,1,5,5,150
2,6,3,2,2,74,1,1,1,11,2,...,2,7,2,2,1,3,1,6,2,170
3,7,4,2,2,33,1,1,1,12,1,...,1,14,2,2,1,1,1,6,0,260
4,5,3,2,2,20,1,2,6,7,2,...,1,6,2,2,2,2,1,5,8,185


In [302]:
rawt['ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'] = rawt['ANY FAMILY MEMBERS OR CLOSE FRIENDS DIED IN LAST 12 MONTHS'].replace(9,2)
rawt['SELF-PERCEIVED CURRENT HEALTH'] = rawt['SELF-PERCEIVED CURRENT HEALTH'].replace(9,3)
rawt['HEIGHT: INCHES']  = rawt['HEIGHT: INCHES'].replace(99,np.median(rawt['HEIGHT: INCHES']))
rawt['HEIGHT: FEET']  = rawt['HEIGHT: FEET'].replace(99,np.median(rawt['HEIGHT: FEET']))

## Combine Heights
rawt['HEIGHT'] = rawt['HEIGHT: FEET']*12 + rawt['HEIGHT: INCHES']


In [303]:
def col_encode(col):
    return pd.get_dummies(rawt[col], prefix = col)

def col_mapped(col):
    return rawt[col].map({1:0,2:1})

def col_normalized(col):
    return MinMaxScaler().fit_transform(rawt[col].values.reshape(-1,1))

In [304]:
rawt['AGE'] = MinMaxScaler().fit_transform(rawt['AGE'].values.reshape(-1,1))
for col in to_be_encoded:
    rawt = pd.concat([rawt, col_encode(col)], axis = 1)

for col in to_be_mapped:
    rawt[col] = col_mapped(col)
    
for col in to_be_normalized:
    rawt[col]  = col_normalized(col)

rawt = rawt.drop(to_be_encoded, axis = 1)
rawt = rawt.drop(['HEIGHT: FEET', 'HEIGHT: INCHES'], axis = 1)

In [305]:
rawt['ONES'] = np.ones(rawt.shape[0])

In [306]:
rawt.head()

,NUMBER OF PERSONS IN HOUSEHOLD,FATHER OF RESPONDENT IN HOUSEHOLD,MOTHER OF RESPONDENT IN HOUSEHOLD,AGE,SEX,PRESENT SITUATION INCLUDES WORKING FULL TIME (35+ HOURS A WEEK),PRESENT SITUATION INCLUDES RETIRED,WORK AT JOB OR BUSINESS IN LAST 12 MONTHS,PERSONALLY RECEIVED FOOD STAMPS IN LAST 12 MONTHS,CURRENTLY COVERED BY MEDICAID,...,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_18,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_19,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_20,TOTAL FAMILY INCOME IN LAST 12 MONTHS (INCLUDING ANY INCOME FROM FOOD STAMPS)_21,SELF-PERCEIVED CURRENT HEALTH_1,SELF-PERCEIVED CURRENT HEALTH_2,SELF-PERCEIVED CURRENT HEALTH_3,SELF-PERCEIVED CURRENT HEALTH_4,SELF-PERCEIVED CURRENT HEALTH_5,ONES
0,0.1250,1,1,0.7375,1,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,1.0
1,0.0000,1,1,0.0750,1,0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,1.0
2,0.1250,1,1,0.7000,0,1,0,1,1,1,...,0,0,0,0,0,0,1,0,0,1.0
3,0.1875,1,1,0.1875,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0,0,1.0
4,0.1250,1,1,0.0250,0,1,1,0,1,1,...,0,0,0,0,0,1,0,0,0,1.0


In [308]:
rawt['TARGET'] = targett

In [309]:
rawt.shape

(13093, 76)

In [310]:
rawt.to_csv('hw1_test.csv')

## Machine Learning using sklearn

In [265]:
from sklearn.linear_model import LogisticRegression

In [275]:
lr = LogisticRegression('l2')

In [276]:
(m, n) = raw.shape
X = raw.ix[:, :n-1]
y = raw.ix[:, n-1] 

In [277]:
X_train = X[:20000]
y_train = y[:20000]
X_test = X[20001:]
y_test = y[20001:]

In [278]:
result = lr.fit(X_train, y_train).score(X_test, y_test)

In [279]:
result

0.8160816081608161

In [280]:
X.shape

(30000, 75)

In [281]:
y_pred = lr.fit(X_train, y_train).predict(X_test)

In [282]:
y_pred[y_pred == 1].sum()

442

In [283]:
y_test.sum()

1921

In [284]:
import a

In [285]:
model = a.Logistic_Regression(0.005, 1, 1000)
score = model.score(X_train, y_train, X_test, y_test)

In [286]:
score

0.80788078807880792

In [288]:
1- y_test.sum()/len(y_test)

0.8078807880788079

In [289]:
from sklearn.metrics import f1_score

In [291]:
f1_score( y_test, y_pred)

0.22175201015658061